In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from vocab import Vocabulary
from env import ConceptData
from create_data import addFile
import tensorflow as tf
import random
import numpy as np
import csv
import re
import os
import create_data
from xml.dom import minidom
import xml.etree.ElementTree as ET
from vocab import Vocabulary
from env2 import ConceptData
from create_data import addFile
import agents
import sender

In [2]:
sender_encoder = sender.Sender(num_options=10, batch_size=32)
sender_only_target = sender.SenderOnlyTarget(batch_size=32)
sender = sender.Sender_LSTM(vocab_size=99, embed_dim=44, num_cells=1, hidden_size=1, max_len=10, see_all_input=True)
receiver_decoder = agents.Receiver(num_options=10)
receiver_encoder = agents.Receiver_LSTM(agent=receiver_decoder, vocab_size=99, embed_dim=44, hidden_size=44, masking_value=99)

In [3]:
vocabulary = Vocabulary()

def addFile(name):
    file_name = name + "_structured_final.xml"
    file = minidom.parse(os.path.join(os.path.join('visa_dataset', 'UK'), file_name))
    concepts = file.getElementsByTagName('concept')

    for concept in concepts:
        vocabulary.addConcept(concept)


addFile("ANIMALS")
addFile("APPLIANCES")
addFile("ARTEFACTS")
addFile("CLOTHING")
addFile("CONTAINER")
addFile("DEVICE")
addFile("FOOD")
addFile("HOME")
addFile("INSTRUMENTS")
addFile("MATERIAL")
addFile("PLANTS")
addFile("STRUCTURES")
addFile("TOOLS")
addFile("TOYS")
addFile("VEHICLES")
addFile("WEAPONS")

for concept in vocabulary.concept_list:
    vocabulary.parseConcept(concept)
    
data = ConceptData(vocabulary, 9, 32)

In [8]:
input_concepts, sender_input, targets, receiver_input = data.getInput()
#print(sender_input.shape)

shape sender input:
(32, 10, 1, 595)
['stereo', 'gopher', 'stork', 'hammer', 'parsley', 'rope', 'crayon', 'giraffe', 'vest', 'mat', 'guppy', 'dresser', 'peas', 'mackerel', 'fawn', 'scissors', 'shelves', 'goldfish', 'falcon', 'emu', 'whistle', 'finch', 'toaster', 'knife', 'bracelet', 'giraffe', 'tiger', 'jeep', 'bag', 'shoes', 'eggplant', 'thermometer']


In [5]:
sender_encoded = sender_encoder(sender_input)
message, logits, entropy, state_h = sender(sender_encoded)
print("message:")
print(message)
print("logits:")
print(logits)
print("entropy:")
print(entropy)
print("state_h:")
print(state_h)

32
input-shape:
(32, 10, 44)
message:
tf.Tensor(
[[62 25 65  2 97 33 67 92 48  0  0  0  0  0  0  0  0  0]
 [15 91 34 83 78 14 73 19 58  0  0  0  0  0  0  0  0  0]
 [15 10 57 13 18  7 47 95 18  0  0  0  0  0  0  0  0  0]
 [71 55 25 68 75  2 17 76 89  0  0  0  0  0  0  0  0  0]
 [50 77 17 92 73 83 45 85 63  0  0  0  0  0  0  0  0  0]
 [35  6 68 56 63 30 65 24 13  0  0  0  0  0  0  0  0  0]
 [84 48 96 44 46 66 34 72 56  0  0  0  0  0  0  0  0  0]
 [41 86 42 52 97 68 69 73 31  0  0  0  0  0  0  0  0  0]
 [33 54 70 95 80 54  9 91 79  0  0  0  0  0  0  0  0  0]
 [39 93 36 46 30 35 17 25 25  0  0  0  0  0  0  0  0  0]
 [26 31  8 37 12 90 91 26 57  0  0  0  0  0  0  0  0  0]
 [65 91 92 85 49 77 20 24 22  0  0  0  0  0  0  0  0  0]
 [86  8 59 72 49 71 38 65 20  0  0  0  0  0  0  0  0  0]
 [98 63 95 32 14 45  1 28 32  0  0  0  0  0  0  0  0  0]
 [38 38 39 38 27 46 80 44 20  0  0  0  0  0  0  0  0  0]
 [66 26  0 56 91 27 78 88 56  0  0  0  0  0  0  0  0  0]
 [90 38 67 94 93 18 17 88 48  0  0  0  

In [14]:

#einer- Dimension rausbekommen
receiver_input = tf.squeeze(tf.convert_to_tensor(receiver_input))
#richtige Reihenfolge der Dimensionen
receiver_input2 = tf.transpose(receiver_input, [1,0,2])
receiver_input2 = receiver_input2.numpy()

sample, log_prob_r, entropy_r, last_hidden = receiver_encoder(message=message, batch_size=32, max_len=10, receiver_input=receiver_input2)



receiver_output = []
for i in range(32):
    receiver_output.append(input_concepts[sample[i]])
    

10
tf.Tensor(
[[[0.47151932 0.47953638 0.4855144  ... 0.4760327  0.63690394 0.5265042 ]
  [0.39563406 0.49870497 0.52212584 ... 0.49496183 0.5041605  0.481551  ]
  [0.5288966  0.49337873 0.5117274  ... 0.4779389  0.46698713 0.5105772 ]
  ...
  [0.4129681  0.5806669  0.5207515  ... 0.5007942  0.49088606 0.4548535 ]
  [0.5402501  0.58027613 0.45185947 ... 0.5785885  0.6012744  0.47745854]
  [0.50839484 0.51072013 0.51874197 ... 0.57118684 0.48306164 0.5086685 ]]

 [[0.4593104  0.5476966  0.50824803 ... 0.5237156  0.44605258 0.45164123]
  [0.48224473 0.54197574 0.48923603 ... 0.51933825 0.57696235 0.52833295]
  [0.45902157 0.4793478  0.538606   ... 0.38455623 0.564518   0.5583493 ]
  ...
  [0.49635386 0.37284824 0.49261048 ... 0.48947936 0.5067419  0.49326357]
  [0.45140296 0.5118193  0.5946147  ... 0.4723551  0.61104184 0.55413824]
  [0.49268833 0.5281216  0.5032553  ... 0.47435203 0.5433748  0.5055043 ]]

 [[0.5209335  0.54072875 0.50613433 ... 0.54759103 0.5274621  0.48367393]
  [0.466